In [ ]:
import sys
import os
from src import app
from gevent import monkey,pywsgi
CURRENT_DIR = os.getcwd()
sys.path.insert(0, CURRENT_DIR)

In [ ]:
monkey.patch_all()
server = pywsgi.WSGIServer(('127.0.0.2', 5000), app)
#app.debug = True
server.serve_forever()
#http://127.0.0.2:5000/

The history saving thread hit an unexpected error (LoopExit('This operation would block forever', <Hub at 0x228b8fee210 select pending=0 ref=0>)).History will not be written to the database.


127.0.0.1 - - [2020-02-24 08:42:46] "GET /bonds HTTP/1.1" 200 7976 0.029077
127.0.0.1 - - [2020-02-24 08:42:46] "GET /static/vendor/fontawesome-free/css/all.min.css HTTP/1.1" 200 57493 0.003008
127.0.0.1 - - [2020-02-24 08:42:46] "GET /static/css/template.css HTTP/1.1" 200 16408 0.003511
127.0.0.1 - - [2020-02-24 08:42:46] "GET /static/css/mycss.css HTTP/1.1" 200 2073 0.001001
127.0.0.1 - - [2020-02-24 08:42:46] "GET /static/css/handsontable.full.min.css HTTP/1.1" 200 38132 0.003007
127.0.0.1 - - [2020-02-24 08:42:46] "GET /static/css/daterangepicker.css HTTP/1.1" 200 8376 0.002507
127.0.0.1 - - [2020-02-24 08:42:46] "GET /static/vendor/jquery/jquery-3.4.1.slim.min.js HTTP/1.1" 200 88469 0.004012
127.0.0.1 - - [2020-02-24 08:42:46] "GET /static/js/template.js HTTP/1.1" 200 1194 0.001504
127.0.0.1 - - [2020-02-24 08:42:46] "GET /static/vendor/bootstrsap4/bootstrap.min.js HTTP/1.1" 200 60338 0.002505
127.0.0.1 - - [2020-02-24 08:42:46] "GET /static/vendor/jquery/popper.min.js HTTP/1.1" 2

127.0.0.1 - - [2020-02-24 08:44:16] "GET /bonds HTTP/1.1" 200 7976 0.007265
127.0.0.1 - - [2020-02-24 08:44:16] "GET /static/vendor/fontawesome-free/css/all.min.css HTTP/1.1" 200 57493 0.003007
127.0.0.1 - - [2020-02-24 08:44:16] "GET /static/css/template.css HTTP/1.1" 200 16479 0.004010
127.0.0.1 - - [2020-02-24 08:44:16] "GET /static/css/mycss.css HTTP/1.1" 200 2073 0.002008
127.0.0.1 - - [2020-02-24 08:44:16] "GET /static/css/handsontable.full.min.css HTTP/1.1" 200 38132 0.002506
127.0.0.1 - - [2020-02-24 08:44:16] "GET /static/vendor/jquery/popper.min.js HTTP/1.1" 200 21585 0.003006
127.0.0.1 - - [2020-02-24 08:44:16] "GET /static/vendor/jquery/jquery-3.4.1.slim.min.js HTTP/1.1" 200 88469 0.004010
127.0.0.1 - - [2020-02-24 08:44:16] "GET /static/css/daterangepicker.css HTTP/1.1" 200 8376 0.002005
127.0.0.1 - - [2020-02-24 08:44:16] "GET /static/vendor/bootstrsap4/bootstrap.min.js HTTP/1.1" 200 60338 0.003510
127.0.0.1 - - [2020-02-24 08:44:16] "GET /static/vendor/daterangepicker/mo

127.0.0.1 - - [2020-02-24 08:44:54] "GET /bonds HTTP/1.1" 200 7976 0.006969
127.0.0.1 - - [2020-02-24 08:44:54] "GET /static/vendor/fontawesome-free/css/all.min.css HTTP/1.1" 200 57493 0.002507
127.0.0.1 - - [2020-02-24 08:44:54] "GET /static/css/template.css HTTP/1.1" 200 16547 0.004010
127.0.0.1 - - [2020-02-24 08:44:54] "GET /static/css/handsontable.full.min.css HTTP/1.1" 200 38132 0.002508
127.0.0.1 - - [2020-02-24 08:44:54] "GET /static/css/mycss.css HTTP/1.1" 200 2073 0.002003
127.0.0.1 - - [2020-02-24 08:44:54] "GET /static/css/daterangepicker.css HTTP/1.1" 200 8376 0.003006
127.0.0.1 - - [2020-02-24 08:44:54] "GET /static/vendor/jquery/jquery-3.4.1.slim.min.js HTTP/1.1" 200 88469 0.004012
127.0.0.1 - - [2020-02-24 08:44:54] "GET /static/vendor/bootstrsap4/bootstrap.min.js HTTP/1.1" 200 60338 0.002506
127.0.0.1 - - [2020-02-24 08:44:54] "GET /static/vendor/jquery/popper.min.js HTTP/1.1" 200 21585 0.002007
127.0.0.1 - - [2020-02-24 08:44:54] "GET /static/vendor/daterangepicker/mo

127.0.0.1 - - [2020-02-24 08:47:58] "GET /bonds HTTP/1.1" 200 7976 0.017037
127.0.0.1 - - [2020-02-24 08:47:58] "GET /static/vendor/fontawesome-free/css/all.min.css HTTP/1.1" 200 57493 0.002507
127.0.0.1 - - [2020-02-24 08:47:58] "GET /static/css/template.css HTTP/1.1" 200 16648 0.003509
127.0.0.1 - - [2020-02-24 08:47:58] "GET /static/css/mycss.css HTTP/1.1" 200 2075 0.002506
127.0.0.1 - - [2020-02-24 08:47:58] "GET /static/css/handsontable.full.min.css HTTP/1.1" 200 38132 0.002509
127.0.0.1 - - [2020-02-24 08:47:58] "GET /static/vendor/jquery/popper.min.js HTTP/1.1" 200 21585 0.002503
127.0.0.1 - - [2020-02-24 08:47:58] "GET /static/vendor/jquery/jquery-3.4.1.slim.min.js HTTP/1.1" 200 88469 0.003008
127.0.0.1 - - [2020-02-24 08:47:58] "GET /static/css/daterangepicker.css HTTP/1.1" 200 8376 0.001503
127.0.0.1 - - [2020-02-24 08:47:58] "GET /static/vendor/bootstrsap4/bootstrap.min.js HTTP/1.1" 200 60338 0.003004
127.0.0.1 - - [2020-02-24 08:47:58] "GET /static/vendor/daterangepicker/mo

127.0.0.1 - - [2020-02-24 08:49:52] "GET /bonds HTTP/1.1" 200 7976 0.007089
127.0.0.1 - - [2020-02-24 08:49:52] "GET /static/vendor/fontawesome-free/css/all.min.css HTTP/1.1" 200 57493 0.003009
127.0.0.1 - - [2020-02-24 08:49:52] "GET /static/css/template.css HTTP/1.1" 200 16651 0.004512
127.0.0.1 - - [2020-02-24 08:49:52] "GET /static/css/mycss.css HTTP/1.1" 200 2075 0.002506
127.0.0.1 - - [2020-02-24 08:49:52] "GET /static/css/handsontable.full.min.css HTTP/1.1" 200 38132 0.002512
127.0.0.1 - - [2020-02-24 08:49:52] "GET /static/vendor/jquery/jquery-3.4.1.slim.min.js HTTP/1.1" 200 88469 0.003009
127.0.0.1 - - [2020-02-24 08:49:52] "GET /static/css/daterangepicker.css HTTP/1.1" 200 8376 0.002507
127.0.0.1 - - [2020-02-24 08:49:52] "GET /static/vendor/bootstrsap4/bootstrap.min.js HTTP/1.1" 200 60338 0.003507
127.0.0.1 - - [2020-02-24 08:49:52] "GET /static/vendor/daterangepicker/moment.min.js HTTP/1.1" 200 51794 0.002005
127.0.0.1 - - [2020-02-24 08:49:52] "GET /static/vendor/jquery/po

In [ ]:
import sys
sys.path.insert(1, r'C:\Users\Kart\jupyter_projects')
import mygit as mg

In [ ]:
folder=r'C:\Users\Kart\jupyter_projects\webtrade'
git_url='git@github.com:Rayveni/web.git'
gt=mg.gitwrapper(folder)
gt.push_commit(git_url,commit_msg='update')

In [ ]:
from jobs import job_sl_bonds
from db_drivers import mongo_manager
cfg={"driver": "mongo", "db_name": "trade", "host": "localhost",
     "port": 27017, "user": "", "user_pswd": "", "mongo_data": "C:\\Users\\DB"}

a=job_sl_bonds(cfg)

In [ ]:
from sys import path
path.append(r"C:\Users\Kart\jupyter_projects\webtrade\webtrade\src")

In [ ]:
from commons import convert_file_size,init_db_manager
from jobs import *
#a=job_sectors(init_db_manager())

In [ ]:
a=job_sl_bonds(init_db_manager())

In [ ]:
a=init_db_manager().get_table('smartlabbondsrus',result='matrix')

In [ ]:
from sys import path
path.append(r"C:\Users\Kart\jupyter_projects\webtrade\webtrade\external_sources")

In [ ]:
from external_sources import smartlab
import pandas as pd
a=smartlab().bonds_info()